In [22]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [23]:
google_api_key=os.getenv("GOOGLE_API_KEY")  # This will load the environment variable from the .env file
# You can now use the GOOGLE_API_KEY in your application

In [24]:
if google_api_key=="":
    print("GOOGLE_API_KEY is not set. Please check your .env file.")
else:
    print("GOOGLE_API_KEY is set successfully.")
    # You can proceed with using the google_api_key in your application logic

GOOGLE_API_KEY is set successfully.


In [39]:
from llama_index.core import (
    VectorStoreIndex,  # Importing VectorStoreIndex from llama_index.core
    SimpleDirectoryReader,  # Importing SimpleDirectoryReader for reading documents from a directory
    StorageContext,  # Importing StorageContext for managing storage
    load_index_from_storage,  # Importing load_index_from_storage to load an index from storage
    ServiceContext,  # Importing ServiceContext for managing service configurations
)
from llama_index.llms.google_genai import GoogleGenAI  # Importing Gemini for LLM functionality
from IPython.display import display, Markdown  # Importing display and Markdown for rendering output in Jupyter notebooks
import google.generativeai as genai  # Importing Google Generative AI for using Google's LLMs
from llama_index.embeddings.google_genai import GoogleGenAIEmbedding  # Importing GeminiEmbedding for embeddings

In [26]:
genai.configure(api_key=google_api_key)  # Configuring Google Generative AI with the API key

In [27]:
for models in genai.list_models():
    print(f"Model: {models.name}")
    print(f"Description: {models.description}")
    print("___________________________________________________________________________________________")

Model: models/embedding-gecko-001
Description: Obtain a distributed representation of a text.
___________________________________________________________________________________________
Model: models/gemini-1.0-pro-vision-latest
Description: The original Gemini 1.0 Pro Vision model version which was optimized for image understanding. Gemini 1.0 Pro Vision was deprecated on July 12, 2024. Move to a newer Gemini version.
___________________________________________________________________________________________
Model: models/gemini-pro-vision
Description: The original Gemini 1.0 Pro Vision model version which was optimized for image understanding. Gemini 1.0 Pro Vision was deprecated on July 12, 2024. Move to a newer Gemini version.
___________________________________________________________________________________________
Model: models/gemini-1.5-pro-latest
Description: Alias that points to the most recent production (non-experimental) release of Gemini 1.5 Pro, our mid-size multimodal 

Text to text generation

In [28]:
documents=SimpleDirectoryReader("../data").load_data()  # Loading data from the specified directory

In [29]:
documents[0].text

'The Marvel Cinematic Universe (MCU) centers on a series of American superhero films produced by Marvel Studios based on characters that appear in publications by Marvel Comics. The MCU is the shared universe in which all of the films are set. The films have been in production since 2007, and in that time Marvel Studios has produced and released 37 films, with at least 8 more in various stages of development. It is the highest-grossing film franchise of all time, having grossed over $31.9 billion at the global box office. This includes Avengers: Endgame, which became the highest-grossing film of all time at the time of its release.\r\n\r\nThe films are written and directed by various individuals and feature large, often ensemble, casts. Marvel Studios president Kevin Feige has produced every film in the franchise, while other Marvel Studios executives have also produced some films alongside Feige, including the studio\'s former CEO Avi Arad for the first two releases. Other individuals

In [30]:
gemini_embed_model=GoogleGenAIEmbedding(model_name="models/embedding-001")

In [31]:
model=GoogleGenAI(model_name="models/gemini-1.5-pro-latest")  # Initializing the Gemini model with the specified model name

In [41]:
from llama_index.core import settings

settings.llm = model
settings.embed_model = gemini_embed_model
settings.chunk_size = 800  # Setting the chunk size for processing documents
settings.chunk_overlap = 100  # Setting the overlap size for chunks

In [44]:
index = VectorStoreIndex.from_documents(documents, embed_model=gemini_embed_model, llm=model)  # Creating an index from the loaded documents

In [45]:
index.storage_context.persist(persist_dir="../storage")  # Persisting the index to the specified storage directory

In [48]:
querry_engine=index.as_query_engine(llm=model)  # Creating a query engine from the index

In [54]:
res=querry_engine.query("What is the capital of France?")  # Example query to test the query engine

In [55]:
res.response

'This question cannot be answered from the given source.\n'

In [56]:
res=querry_engine.query("What is Marvel?")  # Example query to test the query engine

In [57]:
res.response  # Displaying the response from the query engine

'The Marvel Cinematic Universe (MCU) centers on a series of American superhero films produced by Marvel Studios based on characters that appear in publications by Marvel Comics. It is the shared universe in which all of the films are set.\n'